# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
df = pd.read_csv('../Homework_Output/CityWeatherData.csv')
df.drop(columns =('Unnamed: 0'),inplace = True)
df


,City,Country,Latitude,Longitude,Date,Max Temperature (Celsius),Humidity(%),Cloudiness,Wind Speed(km/h)
0,kapaa,US,22.08,-159.32,1587947208,27.00,65,20,5.70
1,georgetown,MY,5.41,100.34,1587947208,26.00,88,20,1.50
2,kodiak,US,57.79,-152.41,1587947208,11.00,43,75,5.10
3,xichang,CN,27.90,102.26,1587947209,12.25,39,100,1.64
4,dingle,PH,11.00,122.67,1587947209,28.37,74,88,5.65
...,...,...,...,...,...,...,...,...,...
581,hailar,CN,49.20,119.70,1587947361,10.60,31,95,5.88
582,skelleftea,SE,64.75,20.95,1587947361,1.00,64,33,2.60
583,roberval,CA,48.52,-72.23,1587947361,3.00,44,20,3.10
584,black forest,US,39.01,-104.70,1587947125,17.22,31,90,4.10


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Configure gmaps.
gmaps.configure(api_key = g_key)

In [8]:
# Store 'Lat' and 'Lng' into  locations 
locations = df[["Latitude", "Longitude"]].astype(float)

# Use humidity as weight
humidity = df["Humidity(%)"].astype(float)

In [11]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [43]:
#   * A max temperature lower than 27 degrees but higher than 21.
loc_temp= (df['Max Temperature (Celsius)'] >21)&(df['Max Temperature (Celsius)'] <27)

#   * Wind speed less than 16 kmph.

loc_wind_speed =df['Wind Speed(km/h)']<16
#   * Zero cloudiness.
loc_cloudiness = df.Cloudiness==0
#   * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
filtered_weather_df = df[loc_temp & loc_wind_speed&loc_cloudiness]
filtered_weather_df

,City,Country,Latitude,Longitude,Date,Max Temperature (Celsius),Humidity(%),Cloudiness,Wind Speed(km/h)
27,namibe,AO,-15.20,12.15,1587947213,22.28,64,0,3.47
32,arraial do cabo,BR,-22.97,-42.03,1587946988,24.07,75,0,5.21
53,san quintin,MX,30.48,-115.95,1587947219,22.90,55,0,5.94
126,adrar,MR,20.50,-10.07,1587947240,26.85,14,0,4.59
173,nizwa,OM,22.93,57.53,1587947252,26.53,22,0,0.37
219,cap malheureux,MU,-19.98,57.61,1587946973,23.33,91,0,2.84
243,miandrivazo,MG,-19.52,45.47,1587947267,23.03,82,0,0.13
253,esna,EG,25.29,32.55,1587947269,24.00,33,0,3.10
320,guerrero negro,MX,27.98,-114.06,1587947288,24.65,50,0,6.98
331,mecca,SA,21.43,39.83,1587947291,24.24,29,0,2.62


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [86]:
hotel_df = pd.DataFrame([df.City, df.Country, df.Latitude,df.Longitude]).transpose()
hotel_df['Hotel Name']=''
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
0,kapaa,US,22.08,-159.32,
1,georgetown,MY,5.41,100.34,
2,kodiak,US,57.79,-152.41,
3,xichang,CN,27.9,102.26,
4,dingle,PH,11,122.67,
...,...,...,...,...,...
581,hailar,CN,49.2,119.7,
582,skelleftea,SE,64.75,20.95,
583,roberval,CA,48.52,-72.23,
584,black forest,US,39.01,-104.7,


In [119]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
#     "location": f"{hotel_df.Latitude},{hotel_df.Longitude}",
    "radius":5000,
    "keyword":"Hotel",
    "type": "Hotel",
    "key": g_key
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    # add keyword to params dict

    params['location'] = f"{row['Latitude']},{row['Longitude']}"

    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()

    # extract results
    results = response['results']

    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        



Retrieving Results for Index 0: kapaa.
Retrieving Results for Index 1: georgetown.
Retrieving Results for Index 2: kodiak.
Retrieving Results for Index 3: xichang.
Retrieving Results for Index 4: dingle.
Missing field/result... skipping.
Retrieving Results for Index 5: yicheng.
Retrieving Results for Index 6: punta arenas.
Retrieving Results for Index 7: albany.
Missing field/result... skipping.
Retrieving Results for Index 8: puerto ayora.
Retrieving Results for Index 9: hilo.
Retrieving Results for Index 10: asyut.
Retrieving Results for Index 11: rikitea.
Retrieving Results for Index 12: tuktoyaktuk.
Retrieving Results for Index 13: bethel.
Retrieving Results for Index 14: yellowknife.
Retrieving Results for Index 15: provideniya.
Missing field/result... skipping.
Retrieving Results for Index 16: muroto.
Retrieving Results for Index 17: mangochi.
Retrieving Results for Index 18: sicuani.
Retrieving Results for Index 19: warmbad.
Missing field/result... skipping.
Retrieving Results f

Retrieving Results for Index 164: tura.
Retrieving Results for Index 165: khandyga.
Retrieving Results for Index 166: lucapa.
Missing field/result... skipping.
Retrieving Results for Index 167: cabedelo.
Retrieving Results for Index 168: boende.
Missing field/result... skipping.
Retrieving Results for Index 169: bengkulu.
Retrieving Results for Index 170: dunedin.
Retrieving Results for Index 171: grindavik.
Retrieving Results for Index 172: atasu.
Missing field/result... skipping.
Retrieving Results for Index 173: nizwa.
Retrieving Results for Index 174: kommunisticheskiy.
Missing field/result... skipping.
Retrieving Results for Index 175: mount gambier.
Retrieving Results for Index 176: inhambane.
Retrieving Results for Index 177: vanderhoof.
Retrieving Results for Index 178: apac.
Retrieving Results for Index 179: uyuni.
Retrieving Results for Index 180: pevek.
Retrieving Results for Index 181: walvis bay.
Retrieving Results for Index 182: florida.
Missing field/result... skipping.


Retrieving Results for Index 326: geraldton.
Retrieving Results for Index 327: svetlaya.
Missing field/result... skipping.
Retrieving Results for Index 328: caravelas.
Retrieving Results for Index 329: soeng sang.
Retrieving Results for Index 330: kalmunai.
Retrieving Results for Index 331: mecca.
Retrieving Results for Index 332: pafos.
Retrieving Results for Index 333: port lincoln.
Retrieving Results for Index 334: ahipara.
Retrieving Results for Index 335: okato.
Missing field/result... skipping.
Retrieving Results for Index 336: kodinsk.
Missing field/result... skipping.
Retrieving Results for Index 337: ouadda.
Missing field/result... skipping.
Retrieving Results for Index 338: asau.
Retrieving Results for Index 339: chilakalurupet.
Retrieving Results for Index 340: moron.
Retrieving Results for Index 341: ous.
Missing field/result... skipping.
Retrieving Results for Index 342: paombong.
Retrieving Results for Index 343: srednekolymsk.
Missing field/result... skipping.
Retrieving

Retrieving Results for Index 486: ulaangom.
Retrieving Results for Index 487: tautira.
Missing field/result... skipping.
Retrieving Results for Index 488: divnomorskoye.
Retrieving Results for Index 489: nueva italia.
Missing field/result... skipping.
Retrieving Results for Index 490: churapcha.
Missing field/result... skipping.
Retrieving Results for Index 491: hare bay.
Missing field/result... skipping.
Retrieving Results for Index 492: obodivka.
Missing field/result... skipping.
Retrieving Results for Index 493: babynino.
Missing field/result... skipping.
Retrieving Results for Index 494: banjar.
Retrieving Results for Index 495: matera.
Missing field/result... skipping.
Retrieving Results for Index 496: ayan.
Missing field/result... skipping.
Retrieving Results for Index 497: gravdal.
Retrieving Results for Index 498: praia.
Retrieving Results for Index 499: nuuk.
Retrieving Results for Index 500: san cristobal.
Retrieving Results for Index 501: sainte-thecle.
Retrieving Results fo

In [120]:
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
0,kapaa,US,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
1,georgetown,MY,5.41,100.34,"Eastern & Oriental Hotel, Penang"
2,kodiak,US,57.79,-152.41,Best Western Kodiak Inn and Convention Center
3,xichang,CN,27.9,102.26,Holiday Inn Express Xichang City Center
4,dingle,PH,11,122.67,Pototan
...,...,...,...,...,...
581,hailar,CN,49.2,119.7,Hulunbeir Hotel
582,skelleftea,SE,64.75,20.95,Quality Hotel Skellefteå Stadshotell
583,roberval,CA,48.52,-72.23,Hôtel Château Roberval
584,black forest,US,39.01,-104.7,Black Forest Lodge


In [121]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [122]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig
# Display figure


Figure(layout=FigureLayout(height='420px'))